In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from matplotlib import pyplot as plt

In [2]:
def load_dataset(file_path):
    """
    Load dataset from file and split into feature matrix X and target variable y
    """
    df = pd.read_csv(file_path)
    X = df.drop('target', axis=1)
    y = df['target']
    return X, y

In [3]:
def preprocess_data(X):
    """
    Encode categorical features and scale numerical features
    """
    for c in X.columns:
        if X[c].dtype == 'object': 
            lbl = LabelEncoder()
            lbl.fit(list(X[c].values))
            X[c] = lbl.transform(X[c].values)
        else:
            scaler = StandardScaler()
            X[c] = scaler.fit_transform(X[c].values.reshape(-1,1))
    return X

In [4]:
def get_models():
    """
    Define a dictionary of machine learning models to evaluate
    """
    models = dict()
    models['Logistic Regression'] = make_pipeline(StandardScaler(), LogisticRegression())
    models['KNN'] = make_pipeline(StandardScaler(), KNeighborsClassifier())
    models['Decision Tree'] = DecisionTreeClassifier()
    models['SVM'] = make_pipeline(StandardScaler(), SVC())
    models['Naive Bayes'] = make_pipeline(StandardScaler(), GaussianNB())
    return models

In [5]:
def evaluate_models(X, y, models):
    """
    Evaluate machine learning models using repeated stratified k-fold cross-validation
    """
    results, names = list(), list()
    for name, model in models.items():
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
        scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        results.append(scores)
        names.append(name)
        print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
    plt.boxplot(results, labels=names, showmeans=True)
    plt.show()